### 參考資料：
1. https://kernc.github.io/backtesting.py/
2. https://havocfuture.tw/blog/python-backtesting-py

### 零、安裝套件

In [ ]:
!pip install yfinance
!pip install backtesting

!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib

### 壹、載入套件

In [14]:
import yfinance as yf # 獲取股市API
import datetime as dt # 計時套件
import pandas as pd # 資料分析套件
from talib import abstract # 計算技術指標套件
from backtesting import Backtest, Strategy # 回測、策略套件
from backtesting.lib import crossover # 找交叉套件
from backtesting.test import SMA, GOOG

### 貳、擷取股價

In [ ]:
# 輸入起始與結束時間
StartTime = dt.datetime(2010, 1, 1)
EndTime = dt.date.today()

# 輸入公司代號
CodeName = '2330.TW'

# 擷取資料
Data = yf.download(CodeName, start = StartTime, end = EndTime)

## 參、回測

In [ ]:
class SMA_Cross(Strategy):
  def init(self):
    self.SMA10 = self.I(SMA, Data['Close'], 10)
    self.SMA40 = self.I(SMA, Data['Close'], 40)

  # 當10日移動平均往上穿越40日移動平均時『買進』，當10日移動平均往下穿越40日移動平均時『賣出』
  def next(self):
      if crossover(self.SMA10, self.SMA40):
          self.buy()
      elif crossover(self.SMA40, self.SMA10):
          self.sell()

bt = Backtest(Data, SMA_Cross, cash = 10000, commission = 0.001425) # 交易手續費0.1425%
output = bt.run()
bt.plot()
print(output)